## World Net similarity
http://www.ccis2k.org/iajit/PDF/Vol.13,%20No.2/7829.pdf

In [3]:
%matplotlib inline
import pandas as pd
import dask.dataframe as daskdf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk import word_tokenize, ngrams
from nltk.corpus import wordnet as wn
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn import metrics

In [2]:
%%time
df = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

Wall time: 12.6 s


In [4]:
wn.synsets('dog')

[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01'),
 Synset('chase.v.01')]

synset1.path_similarity(synset2): Return a score denoting how similar two word senses are, based on the shortest path that connects the senses in the is-a (hypernym/hypnoym) taxonomy. The score is in the range 0 to 1. By default, there is now a fake root node added to verbs so for cases where previously a path could not be found---and None was returned---it should return a value. The old behavior can be achieved by setting simulate_root to be False. A score of 1 represents identity i.e. comparing a sense with itself will return 1.


In [5]:
dog = wn.synset('dog.n.01')
cat = wn.synset('cat.n.01')

hit = wn.synset('hit.v.01')
slap = wn.synset('slap.v.01')


dog.path_similarity(cat)  # doctest: +ELLIPSIS

hit.path_similarity(slap)  # doctest: +ELLIPSIS
wn.path_similarity(hit, slap)  # doctest: +ELLIPSIS
print(hit.path_similarity(slap, simulate_root=False))
print(wn.path_similarity(hit, slap, simulate_root=False))

None
None


In [21]:
print(np.array(df[df.is_duplicate == 1].head(3).question1), 
      np.array(df[df.is_duplicate == 1].head(3).question2))

[ 'Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?'
 'How can I be a good geologist?'
 'How do I read and find my YouTube comments?'] [ "I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?"
 'What should I do to be a great geologist?'
 'How can I see all my Youtube comments?']


In [35]:
t = df[df.is_duplicate == 1].head(3).loc[:,['question1', 'question2']].apply(
    lambda row: tuple([row[0], row[1]]), axis=1)

In [65]:
df[df.is_duplicate == 1].id[0:5]
q1 = df.loc[5, 'question1']
q2 = df.loc[5, 'question2']

In [66]:
regex = re.compile(r'[^\w]', re.IGNORECASE)
lq1 = regex.sub(' ', q1).lower().split()
lq2 = regex.sub(' ', q2).lower().split()

In [79]:
wn.synsets('i')

[Synset('iodine.n.01'),
 Synset('one.n.01'),
 Synset('i.n.03'),
 Synset('one.s.01')]

In [80]:
for w in lq1:
    print(w)
    w = wn.synsets(w)
    print(w)

astrology
[Synset('astrology.n.01')]
i
[Synset('iodine.n.01'), Synset('one.n.01'), Synset('i.n.03'), Synset('one.s.01')]
am
[Synset('americium.n.01'), Synset('master_of_arts.n.01'), Synset('amplitude_modulation.n.01'), Synset('be.v.01'), Synset('be.v.02'), Synset('be.v.03'), Synset('exist.v.01'), Synset('be.v.05'), Synset('equal.v.01'), Synset('constitute.v.01'), Synset('be.v.08'), Synset('embody.v.02'), Synset('be.v.10'), Synset('be.v.11'), Synset('be.v.12'), Synset('cost.v.01')]
a
[Synset('angstrom.n.01'), Synset('vitamin_a.n.01'), Synset('deoxyadenosine_monophosphate.n.01'), Synset('adenine.n.01'), Synset('ampere.n.02'), Synset('a.n.06'), Synset('a.n.07')]
capricorn
[Synset('capricorn.n.01'), Synset('capricornus.n.01'), Synset('capricorn.n.03')]
sun
[Synset('sun.n.01'), Synset('sunlight.n.01'), Synset('sun.n.03'), Synset('sun.n.04'), Synset('sunday.n.01'), Synset('sun.v.01'), Synset('sun.v.02')]
cap
[Synset('cap.n.01'), Synset('cap.n.02'), Synset('detonator.n.01'), Synset('cap.n.04'